In [ ]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
  !pip install unsloth
else:
  !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
  !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
  !pip install --no-deps unsloth
!pip install transformers==4.52.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.7.

In [ ]:
#@ Using FastModel from Unsloth which can load any model:
from unsloth import FastModel
import torch
from transformers import CsmForConditionalGeneration

model, processor=FastModel.from_pretrained(
    model_name='unsloth/csm-1b',
    max_seq_length=2048, #how many tokens can model inputs or attends at a time
    dtype=None, #for gpus, used None for auto detection
    auto_model=CsmForConditionalGeneration,
    load_in_4bit=False, #using false enbles lora 16-bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.4: Fast Mimi patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

unsloth/csm-1b does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


##Lora Adaptors:
- for updating 1 to 10% of all parameters

In [ ]:
model=FastModel.get_peft_model(
    model, #our language model but can be any network module
    r=16, #defines the rank, higher more unique information but higher memory
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj', 'gate_proj'] #where lora should be applied esp attention, mlp
    lora_alpha=16,
    lora_dropout=0, #'0' is optimized
    bias='none', #disables bias tuning
    use_gradient_checkpointing='unsloth', #uses 30% less VRAM, fits 2x larger batch sizes
    random_state=44,
    use_rlora=False, #we support rank stabilized LoRA
    loftq_config=None
)

## Dataset Prep

In [ ]:
from datasets import load_dataset, Audio, Dataset
import os
from transformers import AutoProcessor

processor=AutoProcessor.from_pretrained("unsloth/csm-1b")

raw_ds=load_dataset("MrDragonFox/Elise", split=Train)

#@ Getting the speaker id is important for multi-speaker model and speaker consistency:
speaker_key='source'
if "source" not in raw_ds.column_names and "speaker_id" not in raw_ds.column_names:
  print("Unsloth: No Speaker found")
  new_column=['0'] * len(raw_ds)
elif "source" not in raw_ds.cast_column_nnaes and "speaker" in raw_ds.column_names:
  speaker_key='speaker_id'